
# TP 1 : Theorie des Graphes 1 
#### L3 INFO NEC 2024–2025 <br> Université de Pau et des Pays de l’Adour
###### date: "2024-12-18"

comment ouvrir ce document :
3 possibilités :

- avec jupyter : https://jupyter.org/<br>
 tout a fait possible : https://blog.jupyter.org/interactive-graph-visualization-in-jupyter-with-ipycytoscape-a8828a54ab63

- avec kaggle : https://www.kaggle.com/<br>
 je n'ai pas d'exemple mais il va faloir installer <br>
 les package de manière moins évidente<br>
 ça ressemble a ça : `!pip install interpret-core dash-cytoscape`

- la solution la plus simple : <br>
 avec visual studio code : https://code.visualstudio.com/<br>
 il suffit d'ouvrir le fichier de cliquer sur le bloc tournant a droite ou de lancer<br>
 la première ligne (de code python ou markdown)<br>
 et tout le reste s'installe automatiquement il suffit de cliquer sur installer<br><br>
 il faut cliquer sur le cercle qui tourne<br>
 <img src="./imgs/autopak1.png" alt="etape 1"><br>
 puis cliquer sur<br><br>
 <img src="./imgs/autopak2.png" alt="etape 2" width="300px"><br>
 le reste va suivre il suffit de cliquer<br><br>
 astuce avec visual studio code pour interpreter un bloc de code : shift enter


 



## prérequis

### installer avec pip (sur la machine)

c'est pareil avec Windows, Linux, Macos

##### version de python :

>si vous avez la version python ou python3 il suffit d'ajoute le 3 en fonction de votre version<br>
>et heuresement python fonctionne avec pip et python3 fonctionne avec pip3 (normalement les versions<br>
>actuelles installées sont pip3 avec python3, python n'est plus utilisé donc pip non plus)<br>

#### installer dash_cytoscape avec 2 packages

source : https://manual.cytoscape.org/en/latest/Programmatic_Access_to_Cytoscape_Features_Scripting.html<br>
exemples : https://dash.plotly.com/cytoscape

```sh
pip3 install dash
pip3 install dash-cytoscape
```

#### installer pip3 :

```sh
python3 -m ensurepip --upgrade
```
enlever 3 dans python3 pour les version ultérieures<br>
si problèmes voir : https://pip.pypa.io/en/stable/installation/



## <span style="color:#dd4444">avertissement</span> 

pour la partie graphique il vaut mieux lancer a nouveau tout le code depuis le début<br>
si on veut récupérer la sortie d'un algorithme entécédent car les données créées plus<br>
loin dans le code va altérer des données des graphes ultérieurs. Toutes les sorties graphiques<br>
faites avec __Cytoscape__ et __Dash__ vont être synchronisées sur les sorties de tout le notebook<br>
dans lequel les graphes sont faits.



### importation dans le code

In [1]:
# modules importants pour le(s) tp(s) :

# visualisation des couleurs et autres... (partie graphique)
from dash import Dash, html # type: ignore
import dash_cytoscape as cyto # type: ignore

# partie système
from time import time, sleep
import os, sys
from random import randint

# partie mathématique + structure
import numpy as np # type: ignore
from math import inf, sqrt, cos, sin, tan

### on crée nos classes pour créer une structure de graphe

In [2]:
class V:
    '''représente les informations des sommets (vertex)'''
    def __init__(self, name:str | int, weight=0):
        self.name = name
        self.weight = weight
class Vertex:
    """
    représente les sommets et leurs adjacences dans un graphe
    
    on crée la classe qui s'occupe de toutes les adjacences entre 
    les sommets (vertex/vertices) et leurs autres sommets reliés
    les arrêtes (edges) ne sont pas représentés car c'est la matrice d'adjacence qui s'occupe de ça
    qui est situé dans la classe Graph
    """
    id = 0 # identifiant du sommet 

    def __init__(self, name:str, neighbor:list[V]):
        self.name = name
        self.total_weight = sum([v.weight for v in neighbor]) # poids total du sommet
        self.neighbor = neighbor
        self.vertices_names = [v.name for v in self.neighbor]
        self.degree = len(neighbor)
        self.id = Vertex.id
        Vertex.id += 1

    def __str__(self):
        '''si on affiche Node (print(Node)) renverra ce qui suit'''
        res = f"{self.name} ["
        v_size = len(self.neighbor)
        for i, v in enumerate(self.neighbor):
            res += f"{v.name} w={v.weight}"
            if(i!=v_size-1):
                if v_size>1:
                    res+="," 
                res+=" "
        res+="]"
        return res
    


In [3]:
class Edge:
    '''représente une arrête entre deux sommets/vertex
    manière différente de représenter un graphe'''

    id = 0

    def __init__(self, vfrom:str | int, vto:str | int, weight: int | float):
        self.a = vfrom
        self.b = vto
        self.weight = weight
        self.id = Edge.id
        Edge.id += 1
    
    def __eq__(self, edge:object): # equivallent de ==
        '''compare un edge a un autre objet edge'''
        return self.id == edge.id
    
    def __ne__(self, edge:object): # equivalent de !=
        return not self.__eq__(edge)

In [4]:

class Graph:
    """créée un graphe avec une liste de "Node" en paramètre et un titre"""

    def __init__(self, data:list[Vertex], title:str="default"): # OK
        self.title = title # titre du graphique si utilisé
        self.adj = data # liste d'adjacences
        self.edges = []
        self.verticesn = [] # liste des sommets (seulement les noms)
        self.weight = 0 # poids total du graphe    
        self.degree = 0 # pas encore calculé le degré du graphe
        self.is_complete = True
        for vx in self.adj:
            self.verticesn.append(vx.name)
            self.weight += vx.total_weight
            for v in vx.neighbor:
                self.edges.append(Edge(vx.name, v.name, v.weight))
                self.degree += 1
        self.create_matrix() # matrice des liens entre les noeuds (Edges) (matrice d'adjacences)
        self.port = 8051
        cmd = lambda port : f"netstat -a | grep {port} > /dev/null 2>&1" \
        if os.name != 'nt' else f"netstat -a | findstr {port} > NUL 2>&1"
        while(os.system(cmd(self.port)) != 256): # 256 code pour 'posix' (linux et macos)
            self.port += 1

    def create_matrix(self): # OK
        """créer la matrice d'ajdacence des points
        (fonctionne avec les graphes orienté également)
        """
        self.matrix = [[False for _ in self.adj] for _ in self.adj]
        self.is_oriented=False
        for lin, vis in enumerate(self.adj): # parcours noms des vertex
            for v in vis.neighbor: # parcours des "edges"/sommets/vertex
                self.matrix[lin][self.verticesn.index(v.name)]=True
                # si on trouve que m[i][j] != m[j][i] c'est oriente
                if(not self.is_oriented and vis.name not in self.adj[self.verticesn.index(v.name)].vertices_names):
                    self.is_oriented=True
        iter = range(len(self.matrix))
        # matrice d'un graphe complet avec la liste des sommets actuel
        complete = [[self.matrix[i][j] if i == j else True for i in iter] for j in iter]
        if(self.matrix == complete):
            self.is_complete = True

    def __str__(self): # OK
        """si jamais on print un graph (print(Graph)) c'est executé ici
        affichage au plus simple du graphe avec des caractères"""
        res = ""
        for i, vis in enumerate(self.adj):
            res += f"{i}\t | {vis.name} ["
            v_size = len(vis.neighbor)
            for j in range(v_size):
                res += f"{vis.neighbor[j].name} w={vis.neighbor[j].weight}"
                if(j!=v_size-1):
                    if v_size>1:
                        res += "," 
                    res+=" "
            res += "]\n"
        return res
    
    def _sort_edges(self, edges:list[Edge])->list[Edge]: # OK complexite≈O(n+log(5n))
        """fonction privée a ne pas utiliser (en dehors de la classe)
        algorithme reccursif pour les problèmes de pronfondeur et de performances"""
        if len(edges)<2 :
            return edges
        else:
            pivot = edges[len(edges)//2].weight
            l, m, r = [],[],[] # mineurs, égal, majeurs 
            for e in edges:
                if(e.weight < pivot): l.append(e)
                    # si reccursion sur len(m) ce n'est jamais < 2
                    # et donc (boucle infini) dans certains cas                
                elif(e.weight == pivot): m.append(e) 
                else: r.append(e)
            return self._sort_edges(l)+m+self._sort_edges(r)

    def sort_by_weight(self): # OK
        """trie le graphe par poids croissants(asc)
        on trie chaque arrêtes du graphe
        """
        self.edges = self._sort_edges(self.edges)

    def show_edges(self):
        print([e.a+e.b+" w="+str(e.weight) for e in self.edges])
    
    def render(self, layoutname="breadthfirst"):
        """effectue le rendu du graphe visuellement"""
        unique = f' {time()%1e4:.5}'
        app = Dash(self.title+unique)
        allow_arrows = "linear" # ce style n'autorise pas les flèches
        if self.is_oriented:
            allow_arrows = "bezier" # ce style oui
        custom_style = {
            'width': '100%', 
            'height': '500px',
            "border": "3px white solid",
            "border-radius":"5px",
            "background-color":"#666666",
            "title" : {"background-color":"white"}
        }
        my_styles_sheet = [{
                'selector': 'node',
                'style': {
                    'background-color': '#222222', 
                    'color': 'white',
                    'label': 'data(label)',
                    'font-size': '16px',
                    'text-valign': 'center', 
                    'text-halign': 'center' 
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'width': 2,
                    'target-arrow-shape': "vee",
                    "target-arrow-color": "#4a7cf2",
                    'arrow-scale': 2,
                    'curve-style': allow_arrows
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'label': 'data(weight)',
                    "color": 'white'
                }
            },
        ]
        elems = [] # éléments à afficher (formattés)
        for node in self.adj:
            elems.append({'data': {"id":node.name, "label":node.name}})
        # add edges
        for edge in self.edges:
            elems.append({
                'data': {
                    'source': edge.a, 
                    'target': edge.b, 
                    'weight': edge.weight
                }
            })
        app.layout = html.Div([
            cyto.Cytoscape(
                id='cytoscape'+unique,
                elements=elems,
                layout={'name': layoutname},
                style=custom_style,
                stylesheet=my_styles_sheet
            )
        ])
        print('\nrendu graphique : ')
        print(f"\tégalement ouvert sur la page web : \"localhost:{self.port}\"")
        print(f"\topened too at the web page : \"localhost:{self.port}\"")
        app.run_server(debug=True,port=self.port)

    def resume(self):
        '''crée un résumé du graphe'''
        print("quelques informations sur le graphe : \n")
        print(f"\tdegré: {self.degree}")
        print(f"\tpoids: {self.weight}")
        print(f"\tcomplet : {self.is_complete} (def: si tous les sommets sont reliés)")
        print(f"\tplanaire: voir si les arrêtes se croisent ou non")
        print(f"\t\tnecessite surement de déplacer\n\t\tles noeuds (sur la partie graphique)")


In [5]:
# Tests :

# tester l'efficacité des calculs avec les maths

def generate_unoriented(size = 26):
    f"""génère un graphe non orienté sans boucles (sur un même noeud) de {size} noeuds"""
    # stocakge des adjacences pour compléter les Node dans la liste de Node "nodes"
    res = None
    if(size > 1 and size < 27):
        vertices = [V(chr(i+65)) for i in range(size)]
        adj = [[] for _ in range(size)] 
        for i in range(vertices):
            for _ in range(randint(0,size)): # nb added
                pos = randint(0,25) # index added
                pos += 1 if pos == i else 0
                rand_vertex = Vertex(chr(pos+65),randint(1,400))
                adj[i].append(rand_vertex)
                adj[pos].append(vertices[i])
        res = Graph([Vertex(vertices[i], adj[i]) for i in range(size)])
    else:
        raise Exception(f"not enought or too many vertices to generate unoriented graph (1 < n={size} < 27)")
    return res


# fonction temporaire pour générer une liste de sommets aléatoire (avec edges aléatoire)
# sans avoir de boucle (doc sur le même noeud, paramètre v) exemple: 'v'->'v'
# est un graphe ORIENTE
tmpadata2 = lambda v : [V(chr(i+65), randint(1,400)) for i in range(randint(1,26)) if i != v]
data2 = [Vertex(chr(j+65), tmpadata2(j)) for j in range(26)]
graphe2 = Graph(data2)

print(graphe2)
graphe2.show_edges()
graphe2.sort_by_weight()
graphe2.show_edges()



0	 | A [B w=50, C w=397, D w=183, E w=147, F w=313, G w=373]
1	 | B [A w=294, C w=96]
2	 | C [A w=83, B w=68, D w=382, E w=15, F w=24, G w=179, H w=269, I w=385, J w=318, K w=151, L w=159, M w=269, N w=30, O w=17, P w=105, Q w=222, R w=112, S w=100, T w=18, U w=373, V w=117, W w=53, X w=340]
3	 | D [A w=6, B w=300, C w=178, E w=112, F w=205, G w=5, H w=130, I w=117, J w=354, K w=115, L w=300, M w=200, N w=142, O w=265, P w=315, Q w=267]
4	 | E [A w=323, B w=284, C w=85, D w=202]
5	 | F [A w=38, B w=317, C w=161, D w=393, E w=307, G w=168, H w=311, I w=85, J w=19, K w=232, L w=106, M w=200, N w=62]
6	 | G [A w=182, B w=229, C w=126, D w=36, E w=208, F w=306, H w=64, I w=159, J w=396, K w=361, L w=48, M w=340, N w=390, O w=400]
7	 | H [A w=202, B w=67, C w=294, D w=291, E w=346, F w=126, G w=282, I w=383, J w=260, K w=87, L w=174, M w=359, N w=372, O w=164, P w=33, Q w=42, R w=303, S w=230, T w=398, U w=337, V w=225, W w=322, X w=374, Y w=324]
8	 | I [A w=277, B w=170, C w=74, D w=353, E

In [6]:
# breadthfirst(default) grid circle concentric cose random preset
graphe2.render("grid")


rendu graphique : 
	également ouvert sur la page web : "localhost:8052"
	opened too at the web page : "localhost:8052"


## saisie des données

#### données fournies :

> 1: 2(2), 3(1)<br>
> 2: 1(2), 4(2), 5(3)<br>
> 3: 1(1), 2(3), 4(2)<br>
> 4: 2(2), 3(5), 5(2), 6(4)<br>
> 5: 2(3), 4(2), 6(2)<br>
> 6: 4(2), 5(2)<br>

il y a effectivement une erreur (3->2) sur les données d'origine (il manque donc (2->3))<br>
il y en as une autre ((4->3) et (3->4)) et ((4->6) et (6->4)) les poids ne sont pas les mêmes (5 != 2) et (4 != 2)
donc on rajoute et modifie et ça donne ça :

> 1: 2(2), 3(1)<br>
> 2: 1(2), 4(2), 5(3), 3(3)<br>
> 3: 1(1), 2(3), 4(5)<br>
> 4: 2(2), 3(5), 5(2), 6(4)<br>
> 5: 2(3), 4(2), 6(2)<br>
> 6: 4(4), 5(2)<br>


In [7]:
# rappel format du Vertex : 
# Vertex(nomActuel, [V("nomLié1", poid1),V("nomLié2", poid2),etc...])
data = [
    Vertex("1",[V("2",2),V("3",1)]),
    Vertex("2",[V("1",2),V("4",2),V("5",3),V("3",3)]),
    Vertex("3",[V("1",1),V("2",3),V("4",5)]),
    Vertex("4",[V("2",2),V("3",5),V("5",2),V("6",4)]),
    Vertex("5",[V("2",3),V("4",2),V("6",2)]),
    Vertex("6",[V("4",4),V("5",2)])
]
graphe = Graph(data,"exemple de graphe")
print(np.matrix(graphe.matrix))

gr=Graph([Vertex("1",[])])

# on vérifie que tout est juste
ok = [[False, True, True, False, False, False],
      [True, False, True, True, True, False],
      [True, True, False, True, False, False],
      [False, True, True, False, True, True],
      [False, True, False, True, False, True],
      [False, False, False, True, True, False]]
# on vérifie et renvoie un message si erreur
assert ok == graphe.matrix, "une des valeurs n'est pas vraie"

[[False  True  True False False False]
 [ True False  True  True  True False]
 [ True  True False  True False False]
 [False  True  True False  True  True]
 [False  True False  True False  True]
 [False False False  True  True False]]


## Kruskal

> init: arrêtes d'ordre ascendant de poids (croissant)

on peut utiliser `sort()` qui existe a la fois dans python et dans RStudio<br>
> pour i=1...n-1 des sommets<br>
> &emsp;prendre l'arrête de poids min qui ne fait pas une boucle<br>
> &emsp;et qui n'est pas déja dans la liste des arrêtes que l'on a déja choisit<br>
> fin

### remarque

on a besoin de DFS ou un algorithme avancé que l'on a pas vu en cours<br>
pour savoir si on a une boucle ou non !!!


In [39]:
# on fait de l'héritage car on peut pas 
# rajouter la méthode kruskal comme en swift avec des extensions
class Graph2(Graph):
    """rajoute l'algorithme de Kruskal et un algorithme pour trouver des cycles"""

    inputs = ["list[Vertex]", "Graph"]

    # obligatoire (heritage) 1 seul autorisé en python
    def __init__(self, data:list[Vertex] | Graph, title = "default_title"):
        if(type(data) == list):
            super().__init__(data, title)
        elif(type(data) == Graph):
            # convertit un Graph en Graph2
            super().__init__(data.adj, title)
        else:
            msge = "le contructeur n'accepte pas d'autres type que :"
            msge += f" {", ".join(Graph2.inputs)}"
            raise Exception(f"{msge} (alors que '{type(data)}' est fourni)")

    @staticmethod
    def dfs(graph:Graph)->list[str]:
        '''Fonction DFS pour trouver le chemin le plus court entre 2 sommets
        Depth First Search (algorithme de parcours en profondeur)'''
        visited = set(graph.adj[0].name)
        path = list(graph.adj[0].name)

        if(graph.is_oriented):
            # code here
            pass
        else:
            # code here
            pass

        return path

    @staticmethod
    def get_cycles(edges: list[Edge]) -> list[list[str]]:
        '''Détecte les cycles dans un graphe orienté ou non orienté
        attention cependant sur les graphe orienté ça peut prendre beaucoup de temps'''
        visited = set()  # Pour marquer les sommets visités
        path = []  # suivre le chemin actuel
        cycles = []  # stocker les cycles trouvés
        data : dict[V] = dict()
        for edge in edges:
            if edge.a not in data :
                data[edge.a] = []
            if edge.b not in data :
                data[edge.b] = []
            data[edge.a].append(
                V(edge.b, edge.weight)
            )
        graph = Graph([Vertex(name, data[name]) for name in data])


        # if(graph.is_oriented):
        #     # code a mettre ici
        #     # 1 cycle = minimum 2 sommets et 2 arrêtes de sens opposé entre ces sommets
        #     pass
        # else:
        #     # 1 cycle = minimum 3 sommets et qu'ils soient tous connectés entre eux
        #     if(len(graph.adj)<3):
        #         raise Exception(f"nombre de sommets insuffisants ({len(graph.adj)} < 3)")
        #     tmp = list()
        #     carry = True
        #     current = graph.adj[0].name
        #     can_cycle = set() # ensemble des sommets qui peuvent engendrer un cycle
        #     visited.add(graph.adj[0].name)
        #     while(carry):
        #         # code a mettre ici

        #         # a partir du 3e element on va pouvoir faire des cycles donc la boucle doit changer 
        #         # de comportement a ce moment la
                
        #         if(len(visited) == len(graph.adj)):
        #             carry=False # déja tout visité
        # return cycles


    def kruskal(self,red_tarjan_rule=False)->list[Edge]:
        """renvoie l'arbre couvrant de poids minimal
        applique la règle rouge de tarjan si booléen est True (False par défaut)"""
        # comme vue en cours
        # trie les arrêtes de tout le graphe par poids croissants
        self.sort_by_weight()
        if(len(self.adj)<3):
            raise Exception(f"under minimum required data ({len(self.adj)} vertices < 3)")
        # copie des éléments
        tmp_edges = list(self.edges) 
        tmin = list(tmp_edges.pop(0))
        carry = True # stopper la boucle
        idx = 0
        for edge in self.edges: # pour chaque arretes
            # 1er arrete qui n'est pas dans tmin (forcément la min) et qui fait pas de cycle
            while(carry and idx < len(tmin)): # parcours de tmin
                if(tmin[idx] != edge and not len(Graph2.get_cycles(tmin))):
                    tmin.append(edge)
                idx+=1
            # reset
            carry=True
            idx = 0

    def prim(self):
        '''renvoie un arbre couvrant de poids minimal''' 
        if(not self.is_oriented):
            # on va supprimer des éléments au fur et a mesure pour simplifier
            # et pour avoir plus de performances
            tmp_adj = list(self.adj)
            i2 : int = randint(0,len(self.adj)-1)
            sommets : list[str] = list(self.adj[i2].name)
            # on parcours tous les éléments jusq'a ce que tous les sommets y soient
            j2 : int = 0
            limit = len(self.adj)**3*4/7
            min : int = 0 # securité
            self.poids_total = 0
            while(len(sommets) != len(self.adj)):
                for v_name in sommets:
                    i = self.verticesn.index(v_name)
                    # min = poids du premier element qui n'est pas dans la liste
                    for v in self.adj[i].neighbor:
                        if(v.name not in sommets):
                            min = v.weight
                    i2 = i
                    j2 = 0
                    for j in range(len(self.adj[i].neighbor)):
                        current_v = self.adj[i].neighbor[j]
                        if(current_v.name not in sommets):
                            j2 = j
                            if(current_v.weight < min):
                                min = current_v.weight
                sommets.append(self.adj[i2].neighbor[j2].name)
                self.poids_total += self.adj[i2].neighbor[j2].weight
                if(limit < 0):
                    break
                limit-=1
        return sommets
    
    def prim2(self):
        '''renvoie un arbre couvrant de poids minimal''' 
        limit = len(self.adj)**3*4/7
        # on va supprimer des éléments au fur et a mesure pour simplifier
        # et pour avoir plus de performances        
        tmp_adj = list(self.adj)
        i2 : int = randint(0,len(self.adj)-1)
        sommets : list[str] = list(self.adj[i2].name)        
        if(not self.is_oriented):
            j2 : int = 0
            min : int = 0 # securité
            self.poids_total = 0
            while(len(sommets) != len(self.adj)):
                carry = True
                c1=len(tmp_adj)-1
                min = tmp_adj[c1].neighbor[len(tmp_adj[c1].neighbor)-1].weight
                while(c1 > -1 and carry):
                    c2=len(tmp_adj[c1].neighbor)-1
                    while(c2 > -1 and carry):
                        v=tmp_adj[c1].neighbor[c2]
                        if v.name in sommets :
                            tmp_adj[c1].neighbor.pop(c2)
                            j2 -= 1
                        else:
                            if(v.weight < min):
                                min=v.weight
                                i2 = c1
                                j2 = c2
                        c2-=1
                    c1-=1
                    
                sommets.append(tmp_adj[i2].neighbor[j2].name)
                self.poids_total += tmp_adj[i2].neighbor[j2].weight
                if(limit < 0):
                    break
                limit-=1
        return sommets
    
    def _prim(self, path:list[str]):
        '''fonction reccursive de l'algorithme de primm'''
        if(path == []):
            return self._prim(list(self.adj[randint(0,len(self.adj))]))
        else:
            pass

    def primv():
        pass



        

In [40]:
data = [
    Vertex("1",[V("2",2),V("3",1)]),
    Vertex("2",[V("1",2),V("4",2),V("5",3),V("3",3)]),
    Vertex("3",[V("1",1),V("2",3),V("4",5)]),
    Vertex("4",[V("2",2),V("3",5),V("5",2),V("6",4)]),
    Vertex("5",[V("2",3),V("4",2),V("6",2)]),
    Vertex("6",[V("4",4),V("5",2)])
]
# 3, 4, 6, 5, 4, 4
graphe = Graph2(data) # conversion
print("graphe : ")
print(graphe)
# graph.prim() est une liste on peut la formater :
print(" -> ".join(graphe.prim2()))
print(f"poids de l'arbre couvrant minimal : {graphe.poids_total}")
graphe.render()
# 654321 : poids = 14
# solution 312456: poids = 9



graphe : 
0	 | 1 [2 w=2, 3 w=1]
1	 | 2 [1 w=2, 4 w=2, 5 w=3, 3 w=3]
2	 | 3 [1 w=1, 2 w=3, 4 w=5]
3	 | 4 [2 w=2, 3 w=5, 5 w=2, 6 w=4]
4	 | 5 [2 w=3, 4 w=2, 6 w=2]
5	 | 6 [4 w=4, 5 w=2]



IndexError: list index out of range

methode kruskall en R : 
```R
kruskal <- function(sommets,arretes,poids){
    poids_ord <- sort(poids,index.return=true)
    poids <- poids_ord$x
    index_poids <- poids_ord$ix 
    return poids
}
```

In [144]:
a = [1, 4, 7, 9]
a.remove(1)
a

[4, 7, 9]